In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
import re
from sqlalchemy import create_engine, text
import json
import psycopg2

In [21]:
# MÉTODOS
def extrair_dados_tabela (soup, tag_id):
    produtos = []

    # --- Localiza a tabela com id="tabResult"
    #tabela = soup.find("table", {"id": "tabResult"})
    tabela = soup.find("table", {"id": tag_id})
    
    if tabela:
        linhas = tabela.find_all("tr")
        for linha in linhas:
            colunas = linha.find_all("td")
            if len(colunas) >= 2:
                # Descrição
                descricao_tag = colunas[0].find("span", class_="txtTit")
                descricao = descricao_tag.text.strip() if descricao_tag else ""

                # Código
                codigo_tag = colunas[0].find("span", class_="RCod")
                codigo = ""
                if codigo_tag:
                    match = re.search(r"\d+", codigo_tag.text)
                    if match:
                        codigo = match.group()

                # Extrai o restante do texto do td esquerdo
                info_extra = colunas[0].text

                # Regex para quantidade, unidade e valor unitário
                #qtd_match = re.search(r"Qtde:\s*([\d,\.]+)", info_extra)
                qtd_match = re.search(r"Qtde\.\s*:\s*([\d,\.]+)", info_extra)

                un_match = re.search(r"UN:\s*([A-Za-z]+)", info_extra)
                vl_unit_match = re.search(r"Vl\. Unit\.\s*:\s*([\d,]+)", info_extra)

                quantidade = qtd_match.group(1) if qtd_match else ""
                unidade = un_match.group(1) if un_match else ""
                valor_unitario = vl_unit_match.group(1) if vl_unit_match else ""

                # Valor total
                valor_tag = colunas[1].find("span", class_="valor")
                valor_total = valor_tag.text.strip() if valor_tag else ""

                produtos.append({
                    "descricao_item": descricao,
                    "cod": codigo,
                    "qtde": quantidade,
                    "unid": unidade,
                    "valor_unitario": valor_unitario,
                    "valor_total": valor_total
                })

        df = pd.DataFrame(produtos)
        #print(df)
        return df
    else:
        print("Tabela 'tabResult' não encontrada.")


def extrair_nome_supermercado(soup, tag_info, df):
    # Obtém o nome da empresa e cnpj
    nome = soup.find(id=tag_info).string
    #supplier_name = soup.find(id="u20").string
    df["supermercado"] = nome 
    return df

def extrair_cnpj_endereco (soup, tag_classe, df):
    # Get the CNPJ Number and Address

    # --- Pega todos os <div class="text">
    divs_text = soup.find_all('div', class_= tag_classe)
    #divs_text = soup.find_all('div', class_='text')
    #print(divs_text[1].text) # pode usar o .text para obter os valores

    # Inicializa valores
    cnpj = ""
    endereco = ""

    for i, div in enumerate(divs_text):
        texto = div.text.strip()
        #print(i, text, len(text))
        if 'CNPJ' in texto:
            # Extrai CNPJ com regex
            cnpj_match = re.search(r"\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}", texto)

            if cnpj_match:
                cnpj = cnpj_match.group()
                df["cnpj_emissor"] = cnpj
            
            # Tenta pegar o próximo div como endereço
            if i + 1 < len(divs_text):
                endereco = divs_text[i + 1].text.strip()
                df["endereco"] = endereco
            break

    return df

def extrair_dados_nfe (soup, tag, df): 
    li_info = soup.find(tag) # Localiza o <li> que contém "Número:"
    #li_info = soup.find("li") 
    #info_text = li_info.get_text(" ", strip=True) # Extrai o texto bruto

    dados = {}

    for strong in li_info.find_all("strong"):
        label = strong.text.strip().replace(":", "")
        next_text = strong.next_sibling
        if next_text:
            value = next_text.strip()
            dados[label] = value

    # --- Extrai os valores
    df["numero_nfe"] = dados.get("Número", "")
    df["numero_serie"] = dados.get("Série", "")

    emissao_val = dados.get("Emissão", "")
    # --- Remove qualquer coisa após " -", mantendo apenas a data/hora com fuso
    emissao_limpo = emissao_val.split(" -")[0].strip()
    # Converte para datetime
    emissao_datetime = datetime.strptime(emissao_limpo, "%d/%m/%Y %H:%M:%S%z")

    df["datetime_emissao"] = emissao_datetime

    return df


In [22]:
# --- Configura o Selenium para rodar em modo "headless" (sem abrir janela)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Remova essa linha se quiser ver o navegador
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# --- Inicializa o navegador
driver = webdriver.Chrome(options=chrome_options)

# --- URL do QR Code (exemplo real da nota)
"""
 O que significam os valores após a chave de acesso?
 p = CHAVE|VER|AMBIENTE|TIPO_EMISSOR|DIGEST
1. CHAVE:  Chave de acesso da NFC-e (29250306057223048450650150000775861151017568)
2. VERSÃO=2: Versão do QR Code (ex: 2 = layout da 2ª versão)
3. AMBIENTE=1: produção = 1, Homologação (teste) = 2
4. TIPO_EMISSOR=1:  geralmente 1 = normal, 2 = contingência
5. DIGEST/HASH: DigestValue (resumo criptográfico SHA1 ou SHA256)
O DigestValue é uma assinatura que autentica a URL (como um "hash de verificação" da NFC-e), criado com base nos dados da nota e certificado digital.
 Ele evita alterações manuais ou falsificações na URL.
 """

url_qrcode = "http://nfe.sefaz.ba.gov.br/servicos/nfce/modulos/geral/NFCEC_consulta_chave_acesso.aspx?p=29250306057223048450650150000775861151017568|2|1|1|8DF25CB77517F73F80677AE283AEE7DFCCA38282"
driver.get(url_qrcode)

# --- Aguarda a página carregar completamente
time.sleep(3)  # Pode ajustar dependendo da conexão

# --- Extrai o HTML renderizado
html = driver.page_source
driver.quit()


In [ ]:
# --- Faz o parsing com BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Extrair Dados da Compra "Tabela de itens"
tag_tabela = "tabResult" 
df_raw = extrair_dados_tabela(soup, tag_tabela)

# Extrai nome do supermercado por id
tag_info = "u20"
df = extrair_nome_supermercado(soup, tag_info, df_raw)

tag_id = "text"
df = extrair_cnpj_endereco(soup, tag_id, df)

tag_div = "li"
df = extrair_dados_nfe(soup, tag_div, df)

In [ ]:
# Substituir vírgula por ponto e converter valores numéricos
df["qtde"] = pd.to_numeric(df["qtde"].str.replace(",", "."), errors='coerce')
df["valor_unitario"] = pd.to_numeric(df["valor_unitario"].str.replace(",", "."), errors='coerce')
df["valor_total"] = pd.to_numeric(df["valor_total"].str.replace(",", "."), errors='coerce')

# Converter códigos e números fiscais
df["cod"] = pd.to_numeric(df["cod"], errors='coerce', downcast="integer")
df["numero_nfe"] = pd.to_numeric(df["numero_nfe"], errors='coerce', downcast="integer")
df["numero_serie"] = pd.to_numeric(df["numero_serie"], errors='coerce', downcast="integer")

# Converter colunas textuais para tipo string (novo tipo pandas)
text_cols = [
    "descricao_item", "supermercado", "cnpj_emissor",
    "endereco", "unid"
]

df[text_cols] = df[text_cols].astype("string")




#### Inserir dados no banco de Dados POstgreSQL


# NCM (Nomenclatura Comum do Mercosul)
 é um sistema de categorização de mercadorias usado por Brasil, Argentina, Paraguai e Uruguai desde 1995.

Ele é um sistema que fornece um número único para cada mercadoria usando regras específicas. Esses números representam as mercadorias e são usados em todo comércio internacional.

https://oobj.com.br/legislacao/tabela-ncm-2025/